In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import csv
import pandas as pd
import json

# Load dataset
data = pd.read_csv('nb_filled.csv')

In [16]:
all_cols = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
cont_cols = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
cat_cols = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]

In [17]:
bin_dict = {}
for col in cont_cols:
    result, bins = pd.cut(data[col], bins=10, retbins=True)
    bin_dict[col] = bins
bin_dict

{'age': array([16.927, 24.3  , 31.6  , 38.9  , 46.2  , 53.5  , 60.8  , 68.1  ,
        75.4  , 82.7  , 90.   ]),
 'fnlwgt': array([  10812.58,  159527.  ,  306769.  ,  454011.  ,  601253.  ,
         748495.  ,  895737.  , 1042979.  , 1190221.  , 1337463.  ,
        1484705.  ]),
 'education-num': array([ 0.985,  2.5  ,  4.   ,  5.5  ,  7.   ,  8.5  , 10.   , 11.5  ,
        13.   , 14.5  , 16.   ]),
 'capital-gain': array([  -99.999,  9999.9  , 19999.8  , 29999.7  , 39999.6  , 49999.5  ,
        59999.4  , 69999.3  , 79999.2  , 89999.1  , 99999.   ]),
 'capital-loss': array([  -4.356,  435.6  ,  871.2  , 1306.8  , 1742.4  , 2178.   ,
        2613.6  , 3049.2  , 3484.8  , 3920.4  , 4356.   ]),
 'hours-per-week': array([ 0.902, 10.8  , 20.6  , 30.4  , 40.2  , 50.   , 59.8  , 69.6  ,
        79.4  , 89.2  , 99.   ])}

In [18]:
cat_dict = {}
for col in cat_cols:
    cat_dict[col] = data[col].unique()
cat_dict

{'workclass': array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
        ' Local-gov', ' Self-emp-inc', ' Without-pay', ' Never-worked'],
       dtype=object),
 'education': array([' Bachelors', ' HS-grad', ' 11th', ' Masters', ' 9th',
        ' Some-college', ' Assoc-acdm', ' Assoc-voc', ' 7th-8th',
        ' Doctorate', ' Prof-school', ' 5th-6th', ' 10th', ' 1st-4th',
        ' Preschool', ' 12th'], dtype=object),
 'marital-status': array([' Never-married', ' Married-civ-spouse', ' Divorced',
        ' Married-spouse-absent', ' Separated', ' Married-AF-spouse',
        ' Widowed'], dtype=object),
 'occupation': array([' Adm-clerical', ' Exec-managerial', ' Handlers-cleaners',
        ' Prof-specialty', ' Other-service', ' Sales', ' Craft-repair',
        ' Transport-moving', ' Farming-fishing', ' Machine-op-inspct',
        ' Tech-support', ' Protective-serv', ' Armed-Forces',
        ' Priv-house-serv'], dtype=object),
 'relationship': array([' Not-in-family', ' H

In [19]:
grouped = data.groupby(data.income)
df1 = grouped.get_group(" <=50K")
df2 = grouped.get_group(" >50K")
cond1, cond2 = dict(), dict()

In [20]:
with open('cond1.json') as f:
    cond1 = json.load(f)

In [21]:
with open('cond2.json') as f:
    cond2 = json.load(f)

In [22]:
for col in cat_cols:
    df1[col] = df1[col].replace(cond1[col])
    df2[col] = df2[col].replace(cond2[col])

for col in cont_cols:
    df1[col] = df1[col].replace(cond1[col])
    df2[col] = df2[col].replace(cond2[col])

C:\Users\KHOOSHRIN\AppData\Local\Temp\ipykernel_2440\1810341816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col] = df1[col].replace(cond1[col])
C:\Users\KHOOSHRIN\AppData\Local\Temp\ipykernel_2440\1810341816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].replace(cond2[col])
C:\Users\KHOOSHRIN\AppData\Local\Temp\ipykernel_2440\1810341816.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [23]:
combined_df = pd.concat([df1, df2], axis=0)

In [24]:
combined_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,0.036565,77516,0.127403,13,0.413134,0.132618,0.302711,0.839670,0.607786,2174,0,40,0.906494,<=50K
1,50,0.074162,83311,0.127403,13,0.331817,0.087320,0.291128,0.839670,0.607786,0,0,13,0.906494,<=50K
2,38,0.786247,215646,0.358438,9,0.163362,0.052695,0.302711,0.839670,0.607786,0,0,40,0.906494,<=50K
3,53,0.786247,234721,0.045843,7,0.331817,0.052695,0.291128,0.110424,0.607786,0,0,40,0.906494,<=50K
4,28,0.786247,338409,0.127403,13,0.331817,0.160269,0.033352,0.110424,0.392214,0,0,40,0.002668,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32539,71,0.656526,287372,0.037746,16,0.852582,0.256526,0.751362,0.905915,0.845258,0,0,10,0.933897,>50K
32545,39,0.077371,111499,0.032864,12,0.852582,0.065540,0.097653,0.905915,0.154742,0,0,20,0.933897,>50K
32554,53,0.656526,321865,0.119812,14,0.852582,0.253709,0.751362,0.905915,0.845258,0,0,40,0.933897,>50K
32557,40,0.656526,154374,0.212770,9,0.852582,0.031925,0.751362,0.905915,0.845258,0,0,40,0.933897,>50K


In [25]:
def LREval(k):
    # Separate features and target variable
    X = combined_df.drop('income', axis=1)
    y = combined_df['income']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

    # Initialize logistic regression model
    model = LogisticRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    return accuracy,cm

In [26]:
res = [['K Value','TN','FP','FN','TP','Accuracy','Precision','Recall','F1-Score']]

In [27]:
for i in range(1,11):
    curr_res = []
    acc , confmat = LREval(i)
    print(f"Accuracy: {acc}")
    print(f"K={i}")
    print(f"TP: {confmat[0][0]}\t FN: {confmat[0][1]}\t FP: {confmat[1][0]}\t TN: {confmat[1][1]}")
    acc = (confmat[1][1] + confmat[0][0]) / (confmat[1][1] + confmat[0][1] + confmat[0][0] + confmat[1][0])
    prec = confmat[0][0] / (confmat[1][0] + confmat[0][0])
    rec = confmat[0][0] / (confmat[0][1] + confmat[0][0])
    f1 = 2 / (1 / prec + 1 / rec)
    print(f"Accuracy: {acc}")
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    print(f"F1-Score: {f1}")
    curr_res.append(i)
    curr_res.append(confmat[0][0])
    curr_res.append(confmat[0][1])
    curr_res.append(confmat[1][0])
    curr_res.append(confmat[1][1])
    curr_res.append(acc)
    curr_res.append(prec)
    curr_res.append(rec)
    curr_res.append(f1)
    res.append(curr_res)
    print("\n\n")
    

Accuracy: 0.7918295179601712
K=1
TP: 7854	 FN: 264	 FP: 1973	 TN: 655
Accuracy: 0.7918295179601712
Precision: 0.79922662053526
Recall: 0.967479674796748
F1-Score: 0.8753413207021455



Accuracy: 0.7979713381723432
K=2
TP: 7883	 FN: 266	 FP: 1905	 TN: 692
Accuracy: 0.7979713381723432
Precision: 0.8053739272578668
Recall: 0.9673579580316604
F1-Score: 0.8789652673245248



Accuracy: 0.7955518332402755
K=3
TP: 7865	 FN: 280	 FP: 1917	 TN: 684
Accuracy: 0.7955518332402755
Precision: 0.8040278061746065
Recall: 0.9656230816451811
F1-Score: 0.8774474256707759



Accuracy: 0.7973199329983249
K=4
TP: 7908	 FN: 263	 FP: 1915	 TN: 660
Accuracy: 0.7973199329983249
Precision: 0.8050493739183548
Recall: 0.967812997185167
F1-Score: 0.8789596532177392



Accuracy: 0.794249022892239
K=5
TP: 7872	 FN: 281	 FP: 1930	 TN: 663
Accuracy: 0.794249022892239
Precision: 0.8031014078759436
Recall: 0.9655341592052006
F1-Score: 0.8768588137009188



Accuracy: 0.7973199329983249
K=6
TP: 7829	 FN: 284	 FP: 1894	 TN: 

In [28]:
# open a CSV file for writing
with open('LRResults.csv', 'w', newline='') as file:
    # create a CSV writer object
    writer = csv.writer(file)
    
    # write the data to the file
    for row in res:
        writer.writerow(row)